### Package imports

In [1]:
#import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
#from lightgbm import LGBMClassifier
#from sklearn.neighbors import KNeighborsClassifier

### CSV Imports

In [2]:
df_diet = pd.read_csv("data/diet.csv")
df_recipes = pd.read_csv("data/recipes.csv")
df_requests = pd.read_csv("data/requests.csv")
df_reviews = pd.read_csv("data/reviews.csv", low_memory=False)

### Data Processing

#### 1. Ingredients

##### 1.1. Extract from source

In [3]:
#### Extract ingredientes from source format
df_recipes_ingredients = df_recipes.copy()
cleaned_ingredients_list = []

for i in df_recipes_ingredients.index:
    cleaned_ingredient_string = df_recipes_ingredients['RecipeIngredientParts'][i].replace('c(', '').replace('\\"', '').replace('"', '').replace(')','')
    cleaned_ingredients_list.append({'CleanedIngredients': cleaned_ingredient_string})

cleaned_df_ingredient = pd.DataFrame(cleaned_ingredients_list)

## Cleaning of the quantitites
""" df_recipes_quantity = df_recipes.copy()
cleaned_quantity_list = []

for i in df_recipes_quantity.index:
    cleaned_ingredient_string = df_recipes_quantity['RecipeIngredientQuantities'][i].replace('c(', '').replace('\\"', '').replace('"', '').replace(')','')
    cleaned_quantity_list.append({'CleanedQuantities': cleaned_ingredient_string}) 
    
from fractions import Fraction
def parse_quantity(quantity_str):
    if quantity_str is None:
        return None
    try:
        # Handle mixed numbers (e.g., '1 1/2')
        if ' ' in quantity_str:
            whole, frac = quantity_str.split(' ')
            return float(whole) + float(Fraction(frac))
        # Handle fractions in ranges (e.g., '1/2 - 3/4' or '1/8 - 1/4')
        elif '-' in quantity_str and '/' in quantity_str:
            range_values = re.findall(r'\b\d+\s*\d*/\d+\b', quantity_str)
            range_values = [float(Fraction(val.strip())) for val in range_values]
            return sum(range_values) / len(range_values)
        elif '/' in quantity_str:
            return float(Fraction(quantity_str))
        elif '-' in quantity_str:
            # Handle the case of a range
            range_values = list(map(float, quantity_str.split('-')))
            return sum(range_values) / len(range_values)
        else:
            return float(quantity_str)
    except ValueError:
        # Handle cases where the quantity cannot be converted to a float
        #print(f"Unable to convert '{quantity_str}' to a float")
        return None"""

' df_recipes_quantity = df_recipes.copy()\ncleaned_quantity_list = []\n\nfor i in df_recipes_quantity.index:\n    cleaned_ingredient_string = df_recipes_quantity[\'RecipeIngredientQuantities\'][i].replace(\'c(\', \'\').replace(\'\\"\', \'\').replace(\'"\', \'\').replace(\')\',\'\')\n    cleaned_quantity_list.append({\'CleanedQuantities\': cleaned_ingredient_string}) \n    \nfrom fractions import Fraction\ndef parse_quantity(quantity_str):\n    if quantity_str is None:\n        return None\n    try:\n        # Handle mixed numbers (e.g., \'1 1/2\')\n        if \' \' in quantity_str:\n            whole, frac = quantity_str.split(\' \')\n            return float(whole) + float(Fraction(frac))\n        # Handle fractions in ranges (e.g., \'1/2 - 3/4\' or \'1/8 - 1/4\')\n        elif \'-\' in quantity_str and \'/\' in quantity_str:\n            range_values = re.findall(r\'\x08\\d+\\s*\\d*/\\d+\x08\', quantity_str)\n            range_values = [float(Fraction(val.strip())) for val in range_v

##### 1.2. Create separate columns

In [4]:
# Split the 'CleanedIngredients' column into separate columns
df_split_i = cleaned_df_ingredient['CleanedIngredients'].str.split(', ', expand=True)
"""df_split_q = cleaned_df_quantity['CleanedQuantities'].str.split(', ', expand=True) """
# Rename the columns to include the original index
df_split_i.columns = [f'Ingredient_{i}' for i in range(df_split_i.shape[1])]
"""df_split_q.columns = [f'Quantity_{i}' for i in range(df_split_q.shape[1])] """
# Concatenate the original DataFrame with the new columns
df_result_i = pd.concat([cleaned_df_ingredient, df_split_i], axis=1)
"""df_result_q = pd.concat([cleaned_df_quantity, df_split_q], axis=1) """

# Apply the parse_quantity function to each 'Quantity' column
""" for col in df_result_q.columns:
    df_result_q[col] = df_result_q[col].apply(parse_quantity) """

# Stack the DataFrame and create a new DataFrame
unique_ingredients_df = pd.DataFrame(df_result_i.stack().reset_index(drop=True), columns=['Ingredients'])
#unique_ingredients_df['Diet']='vegan'

##### 1.3. Manual diet mapping

In [5]:
""" # Define keywords for different diet types
diet_keywords = {
    'Vegetarian': ['milk', 'butter', 'egg','buttermilk biscuits', 'ricotta cheese', 'custard', 'clarified butter',
    'queso fresco', 'gouda cheese', 'brie cheese', 'halloumi cheese','cheese','chocolate'
    'caramel', 'creme brulee', 'eggnog', 'mango lassi', 'creamy dressings','eggs'
    'hollandaise sauce', 'anchovy paste', 'pâté',
    'scallops', 'clam chowder', 'crab cakes', 'crème fraîche', 'gravlax', 
    'gelato', 'honey mustard', 'miso butter', 'spicy mayo',
    'burrata cheese', 'whipped honey butter',  'deviled eggs', 
    'pumpkin cheesecake', 'goat cheese-stuffed figs', 'crispy prosciutto',
    'chocolate mousse', 'garlic butter mushrooms', 'pesto pasta with parmesan'
    'sour cream', 'cottage cheese', 'mayonnaise', 'whipped cream',
    'ghee', 'paneer', 'parmesan cheese', 'mozzarella cheese',
    'feta cheese', 'goat cheese', 'blue cheese', 'cream cheese',
    'buttermilk', 'marshmallows', 'white chocolate', 'casein',
    'whey', 'honeycomb', 'royal jelly', 'propolis',
    'egg noodles', 'ravioli with cheese filling', 'honey mustard dressing',
    'popcorn with butter', 'french onion dip', 'onion soup with cheese',
    'chocolate-covered pretzels', 'caramel popcorn', 'milk chocolate',
    'cheese fondue', 'risotto with parmesan', 'strawberries and cream',
    'egg salad', 'cheese soufflé', 'cheesecake', 'honey butter',
    'buttermilk pancakes', 'mango lassi with yogurt', 'cream-based sauces',
    'buttery croissants', 'garlic butter shrimp', 'lobster with butter',
    'baked brie with honey and nuts'],


    'Omnivore': ['beef', 'chicken', 'fish', 'beef', 'chicken', 'pork', 'lamb', 'veal', 'duck', 'turkey', 'rabbit',
    'bacon', 'sausage', 'ham', 'salami', 'pepperoni', 'prosciutto',
    'fish', 'shrimp', 'lobster', 'crab', 'clams', 'mussels', 'oysters',
    'anchovies', 'sardines', 'herring', 'tuna', 'caviar', 'escargot',
    'bone broth', 'lard', 'schmaltz', 'animal fat', 'ghee']

    # Add more diet types and their associated keywords as needed
}

# Function to determine diet type based on keywords
def infer_diet(ingredients):
    omnivore_keywords = diet_keywords.get('Omnivore', [])
    vegetarian_keywords = diet_keywords.get('Vegetarian', [])

    # Check for omnivore keywords first
    if any(keyword in ingredients.lower() for keyword in omnivore_keywords):
        return 'Omnivore'
    # Check for vegetarian keywords if not omnivore
    elif any(keyword in ingredients.lower() for keyword in vegetarian_keywords):
        return 'Vegetarian'
    else:
        return 'Vegan'  # Default to Vegan if no diet type is inferred
    
# Apply the function to create the 'Diet' column
cleaned_df_ingredient['meat_vegan_veggie'] = cleaned_df_ingredient['CleanedIngredients'].apply(infer_diet)
cleaned_df_ingredient['RecipeId'] = df_recipes['RecipeId']
cleaned_df_ingredient['Omnivore'] = 0
cleaned_df_ingredient['Vegeterian'] = 0
cleaned_df_ingredient['Vegan'] = 0
cleaned_df_ingredient.loc[cleaned_df_ingredient['meat_vegan_veggie'] == 'Omnivore', 'Omnivore'] = 1
cleaned_df_ingredient.loc[cleaned_df_ingredient['meat_vegan_veggie'] == 'Vegan', ['Omnivore','Vegeterian','Vegan']] = 1
cleaned_df_ingredient.loc[cleaned_df_ingredient['meat_vegan_veggie'] == 'Vegeterian', ['Vegeterian','Omnivore']] = 1 """

" # Define keywords for different diet types\ndiet_keywords = {\n    'Vegetarian': ['milk', 'butter', 'egg','buttermilk biscuits', 'ricotta cheese', 'custard', 'clarified butter',\n    'queso fresco', 'gouda cheese', 'brie cheese', 'halloumi cheese','cheese','chocolate'\n    'caramel', 'creme brulee', 'eggnog', 'mango lassi', 'creamy dressings','eggs'\n    'hollandaise sauce', 'anchovy paste', 'pâté',\n    'scallops', 'clam chowder', 'crab cakes', 'crème fraîche', 'gravlax', \n    'gelato', 'honey mustard', 'miso butter', 'spicy mayo',\n    'burrata cheese', 'whipped honey butter',  'deviled eggs', \n    'pumpkin cheesecake', 'goat cheese-stuffed figs', 'crispy prosciutto',\n    'chocolate mousse', 'garlic butter mushrooms', 'pesto pasta with parmesan'\n    'sour cream', 'cottage cheese', 'mayonnaise', 'whipped cream',\n    'ghee', 'paneer', 'parmesan cheese', 'mozzarella cheese',\n    'feta cheese', 'goat cheese', 'blue cheese', 'cream cheese',\n    'buttermilk', 'marshmallows', 'whit

##### 1.4. Automatic column diet mapping

In [6]:
""" df_temp = df_result_i.copy()
df_temp = df_temp.drop('CleanedIngredients',axis=1)
unique_ingredients_list = pd.unique(df_temp.values.flatten()).tolist() """

" df_temp = df_result_i.copy()\ndf_temp = df_temp.drop('CleanedIngredients',axis=1)\nunique_ingredients_list = pd.unique(df_temp.values.flatten()).tolist() "

In [7]:
""" cleaned_df_ingredient = df_result_i.copy()
# Fill NaN values with an empty string
cleaned_df_ingredient['CleanedIngredients'] = cleaned_df_ingredient['CleanedIngredients'].fillna('')

# Loop through the ingredient names and create columns
for ingredient in unique_ingredients_list:
    try:
        cleaned_df_ingredient[ingredient] = cleaned_df_ingredient['CleanedIngredients'].apply(lambda x: 1 if x is not None and ingredient in str(x) else 0)
    except Exception as e:
        print(f"Error processing ingredient '{ingredient}': {e}")

cleaned_df_ingredient = cleaned_df_ingredient.filter(regex='^(?!Ingredient)')
cleaned_df_ingredient['RecipeId'] = df_recipes['RecipeId']
cleaned_df_ingredient = cleaned_df_ingredient.drop('CleanedIngredients', axis=1) """

' cleaned_df_ingredient = df_result_i.copy()\n# Fill NaN values with an empty string\ncleaned_df_ingredient[\'CleanedIngredients\'] = cleaned_df_ingredient[\'CleanedIngredients\'].fillna(\'\')\n\n# Loop through the ingredient names and create columns\nfor ingredient in unique_ingredients_list:\n    try:\n        cleaned_df_ingredient[ingredient] = cleaned_df_ingredient[\'CleanedIngredients\'].apply(lambda x: 1 if x is not None and ingredient in str(x) else 0)\n    except Exception as e:\n        print(f"Error processing ingredient \'{ingredient}\': {e}")\n\ncleaned_df_ingredient = cleaned_df_ingredient.filter(regex=\'^(?!Ingredient)\')\ncleaned_df_ingredient[\'RecipeId\'] = df_recipes[\'RecipeId\']\ncleaned_df_ingredient = cleaned_df_ingredient.drop(\'CleanedIngredients\', axis=1) '

##### 1.5. Improved mapping

In [8]:
def read_classification_csv(file_path):
    df = pd.read_csv(file_path)
    return df.set_index('Ingredient').to_dict()['Diet Classification']

def classify_dietary_category(ingredients_list, classification_dict):
    ingredients = [ingredient.strip() for ingredient in ingredients_list.split(",")]
    if any(classification_dict.get(ingredient, 'Vegan') == 'Omnivore' for ingredient in ingredients):
        return 'Omnivore'
    elif any(classification_dict.get(ingredient, 'Vegan') == 'Vegetarian' for ingredient in ingredients):
        return 'Vegetarian'
    else:
        return 'Vegan'

# Read the classification CSV file
csv_file_path = "data/classified_ingredients.csv"  # Adjust the path as needed
classification_dict = read_classification_csv(csv_file_path)

# Apply the classification function to the 'CleanedIngredients' column of df_result_i
df_result_i['Diet Classification'] = df_result_i['CleanedIngredients'].apply(lambda x: classify_dietary_category(x, classification_dict))

pd.set_option('display.max_colwidth', None)
cleaned_df_ingredient = df_result_i[['CleanedIngredients','Diet Classification']].copy()
cleaned_df_ingredient = cleaned_df_ingredient.filter(regex='^(?!Ingredient)')
cleaned_df_ingredient['RecipeId'] = df_recipes['RecipeId']
cleaned_df_ingredient = cleaned_df_ingredient.drop('CleanedIngredients', axis=1)

##### 1.6. Merge data

In [23]:
# Load data to modify imports
recipes = df_recipes.copy()
reviews = df_reviews.copy()
diet = df_diet.copy()
requests = df_requests.copy()

df_unprocessed = pd.merge(reviews, requests, on=['AuthorId','RecipeId'], how='inner')
df_unprocessed = pd.merge(df_unprocessed, recipes, on='RecipeId', how='left')
df_unprocessed = pd.merge(df_unprocessed, cleaned_df_ingredient, on='RecipeId', how='left')
df_unprocessed = pd.merge(df_unprocessed, diet, on='AuthorId', how='left')

merged_data = df_unprocessed.copy()
# Test DFs
prediction_on_revies = reviews.dropna(subset=["TestSetId"])
#reviews_got = reviews.dropna(subset=["Like"])

##### 1.7. Modify columns

In [24]:
# Add new column is_cooking_time_within_expectation
merged_data['is_cooking_time_within_expectation'] = (merged_data['CookTime'] + merged_data['PrepTime']) > merged_data['Time']
# Data preprocessing
# Drop irrelevant columns and handle missing values
merged_data = merged_data.drop(['Name',
                                'RecipeYield',
                                'CookTime',
                                'PrepTime',
                                'Time',
                                'RecipeServings',
                               # 'Age',
                               # 'CleanedIngredients',
                                'TestSetId',
                                'AuthorId',
                                'RecipeId',
                                'RecipeIngredientQuantities',
                                'RecipeIngredientParts',
                                'Rating'], 
                                axis=1)

# Assuming merged_data is your DataFrame
data_to_encode = merged_data[['Diet', 
                              'RecipeCategory', 
                              'LowSugar', 
                              'HighProtein',
                              'Diet Classification'
                              ]].copy()

# Create OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)

# Fit and transform the selected columns
data_encoded = onehot_encoder.fit_transform(data_to_encode)

# Create a DataFrame with the one-hot encoded columns
data_encoded_df = pd.DataFrame(data_encoded, 
                               columns=onehot_encoder.get_feature_names_out(data_to_encode.columns))

# Drop the original columns from the DataFrame
merged_data = merged_data.drop(columns=['Diet', 
                                        'RecipeCategory', 
                                        'LowSugar', 
                                        'HighProtein',
                                        'Diet Classification'
                                        ])

# Concatenate the one-hot encoded columns with the original DataFrame
merged_data = pd.concat([merged_data, 
                         data_encoded_df], axis=1)

c:\ProgramData\Anaconda3\envs\baml-venv\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


##### 1.9 Check columns

In [25]:
# Column check on 
merged_data.columns

Index(['Like', 'HighCalories', 'LowFat', 'HighFiber', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'Age', 'is_cooking_time_within_expectation', 'Diet_Omnivore',
       'Diet_Vegan', 'Diet_Vegetarian', 'RecipeCategory_Beverages',
       'RecipeCategory_Bread', 'RecipeCategory_Breakfast',
       'RecipeCategory_Lunch', 'RecipeCategory_One dish meal',
       'RecipeCategory_Other', 'RecipeCategory_Soup', 'LowSugar_0',
       'LowSugar_Indifferent', 'HighProtein_Indifferent', 'HighProtein_Yes',
       'Diet Classification_Omnivore', 'Diet Classification_Vegan',
       'Diet Classification_Vegetarian'],
      dtype='object')

### Model

##### 2.1. Define features

In [26]:
# Define features and target variable
X_predict = merged_data.copy()
X_training = merged_data.dropna()
label_encoder = LabelEncoder()
X_training['Like'] = label_encoder.fit_transform(X_training['Like'])
X = X_training.drop('Like', axis=1)
y = X_training['Like']

C:\Users\Asus\AppData\Local\Temp\ipykernel_20580\4161558566.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_training['Like'] = label_encoder.fit_transform(X_training['Like'])


#### Train best model

In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Train and evaluate models
results = {}

best_model = None
best_balanced_acc = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    results[name] = {
        'Balanced Accuracy': balanced_acc,
        'Classification Report': report
    }

    # Save the best model based on balanced accuracy
    if balanced_acc > best_balanced_acc:
        best_model = model
        best_balanced_acc = balanced_acc

# Display results
for name, result in results.items():
    print(f'{name}:')
    print(f'Balanced Accuracy: {result["Balanced Accuracy"]:.8f}')
    print('Classification Report:')
    print(result['Classification Report'])
    print('\n' + '-'*50 + '\n')

private_test_predictions = pd.DataFrame({'True_Label': y_test, 'Predicted_Label': y_pred})
private_test_predictions.to_csv('data/Results.csv', index=False)

Random Forest:
Balanced Accuracy: 0.67598970
Classification Report:
{'0': {'precision': 0.9077312218189816, 'recall': 0.9818592755605782, 'f1-score': 0.9433412383210948, 'support': 16813.0}, '1': {'precision': 0.7637490317583269, 'recall': 0.37012012012012013, 'f1-score': 0.4986093552465234, 'support': 2664.0}, 'accuracy': 0.8981876058941315, 'macro avg': {'precision': 0.8357401267886542, 'recall': 0.6759896978403491, 'f1-score': 0.7209752967838091, 'support': 19477.0}, 'weighted avg': {'precision': 0.888037811420995, 'recall': 0.8981876058941315, 'f1-score': 0.8825122740806749, 'support': 19477.0}}

--------------------------------------------------

XGBoost:
Balanced Accuracy: 0.74290904
Classification Report:
{'0': {'precision': 0.9269140869961804, 'recall': 0.9670493070838042, 'f1-score': 0.9465564417535076, 'support': 16813.0}, '1': {'precision': 0.7138429752066116, 'recall': 0.5187687687687688, 'f1-score': 0.6008695652173913, 'support': 2664.0}, 'accuracy': 0.9057349694511475, 'm

# Predictions

#### Generate predictions

In [ ]:
# Prediciton
# Make predictions on the test set
X_predict = X_predict.drop('Like', axis=1)
y_pred = model.predict(X_predict)

### Export Predictions

In [ ]:
test_predictions = pd.DataFrame({'id': prediction_on_revies["TestSetId"].astype(int), 'prediction': y_pred})
print(test_predictions)
test_predictions.to_csv('data/Results.csv', index=False)